<a href="https://colab.research.google.com/github/Pardhu156/Movie-Recommendation-System/blob/main/Stacked_AutoEncoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

## Data Preprocessing

In [ ]:
training_set=pd.read_csv('ml-100k/u1.base',delimiter='\t')
training_set=np.array(training_set,dtype='int')
test_set=pd.read_csv('ml-100k/u1.test',delimiter='\t')
test_set=np.array(test_set,dtype='int')

In [ ]:
nb_users=int(max(max(training_set[ : ,0]),max(test_set[ : ,0])))
nb_movies=int(max(max(training_set[ : ,1]),max(test_set[ : ,1])))

In [ ]:
def convert(data):
  new_data=[]
  for id_user in range(1,nb_users+1):
    ratings=np.zeros(nb_movies)
    id_movie=data[ : ,1][data[ : ,0]==id_user]
    id_rating=data[ : ,2][data[ : ,0]==id_user]
    ratings[id_movie-1]=id_rating
    new_data.append(list(ratings))
  return new_data

In [ ]:
training_set=convert(training_set)
test_set=convert(test_set)

In [ ]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

## Architecture of Neural Network

In [ ]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE,self).__init__()
    self.fc1=nn.Linear(nb_movies,20)
    self.fc2=nn.Linear(20,10)
    self.fc3=nn.Linear(10,20)
    self.fc4=nn.Linear(20,nb_movies)
    self.activation=nn.Sigmoid()
  def forward(self,x):
    x=self.activation(self.fc1(x))
    x=self.activation(self.fc2(x))
    x=self.activation(self.fc3(x))
    x=self.fc4(x)
    return x
sae=SAE()
criterion=nn.MSELoss()
optimizer=optim.RMSprop(sae.parameters(),lr=0.01,weight_decay=0.5)

In [ ]:
nb_epochs=200
for epoch in range(1,nb_epochs+1):
  train_loss=0
  s=0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target=input.clone()
    if torch.sum(input.data>0)>0:
      output=sae.forward(input)
      target.requires_grad=False
      output[target==0]=0
      loss=criterion(output,target)
      loss.backward() #decides wheather to increase or decrease weights
      mean_corrector=nb_movies/float(torch.sum(target.data>0)+1e-10)
      train_loss=train_loss+np.sqrt(loss.data*mean_corrector)
      optimizer.step() #decides the value to be changed
      s=s+1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

/tmp/ipython-input-1048537761.py:15: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  train_loss=train_loss+np.sqrt(loss.data*mean_corrector)


epoch: 1 loss: tensor(1.7724)
epoch: 2 loss: tensor(1.0966)
epoch: 3 loss: tensor(1.0534)
epoch: 4 loss: tensor(1.0384)
epoch: 5 loss: tensor(1.0308)
epoch: 6 loss: tensor(1.0268)
epoch: 7 loss: tensor(1.0239)
epoch: 8 loss: tensor(1.0218)
epoch: 9 loss: tensor(1.0208)
epoch: 10 loss: tensor(1.0195)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0182)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0175)
epoch: 15 loss: tensor(1.0173)
epoch: 16 loss: tensor(1.0168)
epoch: 17 loss: tensor(1.0167)
epoch: 18 loss: tensor(1.0167)
epoch: 19 loss: tensor(1.0165)
epoch: 20 loss: tensor(1.0160)
epoch: 21 loss: tensor(1.0159)
epoch: 22 loss: tensor(1.0158)
epoch: 23 loss: tensor(1.0158)
epoch: 24 loss: tensor(1.0155)
epoch: 25 loss: tensor(1.0156)
epoch: 26 loss: tensor(1.0155)
epoch: 27 loss: tensor(1.0154)
epoch: 28 loss: tensor(1.0150)
epoch: 29 loss: tensor(1.0130)
epoch: 30 loss: tensor(1.0118)
epoch: 31 loss: tensor(1.0095)
epoch: 32 loss: tensor(1.0093)
epoch: 33 loss: t

## Testing the SAE

In [ ]:
test_loss=0
s=0.
for id_user in range(nb_users):
  input=Variable(training_set[id_user]).unsqueeze(0)
  target=Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data>0)>0:
    output=sae.forward(input)
    target.requires_grad=False
    output[target==0]=0
    loss=criterion(output,target)
    mean_corrector=nb_movies/float(torch.sum(target.data>0)+1e-10)
    test_loss=test_loss+np.sqrt(loss.data*mean_corrector)
    s=s+1.
print('epoch: '+str(epoch)+' loss: '+str(test_loss/s))

/tmp/ipython-input-169854596.py:12: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  test_loss=test_loss+np.sqrt(loss.data*mean_corrector)


epoch: 200 loss: tensor(0.9571)
